# Demo notebook

This is a demo notebook to perform query in CYPHER directly from Python.

In [169]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable
import streamlit as st
class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()


    def find_pattern(self, query_string):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_and_return_pattern, query_string)
            for row in result:
                .....TODO...... ##########<----- here you have to change with the data you inserted into the query.
            #for example if we have RETURN A.match_id, P, count(c) as cnt you can access to the returned values using 
            # row["A.match_id"], row["P"] and row["cnt"] respectively


    @staticmethod
    def _find_and_return_pattern(tx, query_string):
        result = tx.run(query_string)
        return [row for row in result]

In [190]:
#this is a useful function to automatically write pattern matching query
"""
For example cypherify(ABAC, "ITALY") returns
MATCH p=(A:ITALY)-[p0:PASS]->(B:ITALY)-[p1:PASS]->(A:ITALY)-[p2:PASS]->(C:ITALY)
WHERE p0.order + 1 = p1.order and p1.order + 1 = p2.order and p0.possession = p1.possession 
and p1.possession = p2.possession and p0.match_id = p1.match_id and p1.match_id = p2.match_id 
and A.name <> C.name and A.name <> B.name and B.name <> C.name
RETURN COUNT(p)

Then you can modify in to get whatever you want (ax you want to draw a heatmap of passing motifs, 
then you can return pass.location and pass.end_location for every pass)
"""

def cypherify(string, team = None):
    letters = list(string)

    if team:
        query = "MATCH p=(A:"+team+")"
        for i in range(len(string) - 1):
            query += "-[p" + str(i) + ":PASS]->(" + letters[i + 1] + ":"+team+")"
    else:
        query = "MATCH (A)"
        for i in range(len(string) - 1):
            query += "-[p" + str(i) + ":PASS]->(" + letters[i + 1] + ")"

    query += "\nWHERE "

    #correct order
    first = True
    for i in range(len(string) - 2):
        if first:
            query += "p" + str(i) + ".order + 1 = p" + str(i + 1) + ".order"
            first = False
        else:
            query += " and p" + str(i) + ".order + 1 = p" + str(i + 1) + ".order"

    #same possession
    first = True
    for i in range(len(string) - 2):
        if first:
            query += " and p" + str(i) + ".possession = p" + str(i + 1) + ".possession"
            first = False
        else:
            query += " and p" + str(i) + ".possession = p" + str(i + 1) + ".possession"

    #same match
    first = True
    for i in range(len(string) - 2):
        if first:
            query += " and p" + str(i) + ".match_id = p" + str(i + 1) + ".match_id"
            first = False
        else:
            query += " and p" + str(i) + ".match_id = p" + str(i + 1) + ".match_id"

    #different players, we have to check all the Nc2 possible combinations (N = number of distinct players)
    unorderedPairGenerator = ((x, y) for x in set(letters) for y in set(letters) if y > x)
    
    if string != "AB": #special case, if we are matching AB we don't have anything before so we avoid the and at the beginning 
        query += " and " + " and ".join([x + ".name <>" + " " +y + ".name" for x, y in list(unorderedPairGenerator)])
    else:
        query +=  " and ".join([x + ".name <>" + " " +y + ".name" for x, y in list(unorderedPairGenerator)])
    
    
    query += "\n RETURN COUNT(p)"
    return query

In [ ]:
#parameters to connect to neo4j server
uri = "neo4j+s://00e145e7.databases.neo4j.io:7687"
user = "soccer_analytics"
password = "night-candle-miracle-nickel-declare-32"
app = App(uri, user, password)

In [192]:
"""
Typically to match a pattern I use these 2 functions
"""
query = cypherify(pattern, team)
v = app.find_pattern(query)

    